# 🦜🔗 Langchain Demo

Welcome! This example will give you a first look at langchain. 

In [7]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Load Environment

Load ENV Variables from .env file

needed ENV vars:

```
# OpenAI Hosted Variante
OPENAI_API_KEY=<required>
OPENAI_ORGANIZATION=""
OPENAI_MODEL="gpt-4-0125-preview"

# Azure Hosted Variante
AZURE_OPENAI_API_KEY=<required>
AZURE_OPENAI_ENDPOINT=<required>
AZURE_OPENAI_API_VERSION="2023-12-01-preview"
AZURE_OPENAI_DEPLOYMENT_NAME=<required>
``` 

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Erster Test

### OpenAI-Modelle
ChatGPT oder auch jedes andere LLM benutzen ist relativ einfach mit Langchain

In diesen Test nutzen wir das neueste "gpt-4-turbo" Model - mögliche Large Language Modelle von OpenAI sind:
- `gpt-35-turbo`  Das günstigste und am weitesten verbreitete Modell
- `gpt-4`  Das neue und bessere GPT Modell
- `gpt-4-turbo`  Turbo-variante von gpt-4 (günstiger, schneller, kleinere maximale Länge des Text-Outputs)
- `gpt-4-vision`  Ein "multimodales" Modell, welches auch auf Bilder trainiert wurde.

OpenAI trainiert diese Versionen laufend neu, was dazu führen kann, das Anfragen an das LLM plötzlich andere Antworten geben.
Möchte man dies verhindern, kann man seine Applikation auf einen Snapshot (z.b. gpt-4-0613) festsetzen.
Dies ist insbesondere wichtig, wenn die Applikation vom Output des LLM bestimmte Strukturen erwartet, beispielsweise eine bestimmte XML-Syntax o.Ä.

OpenAI-Modelle werden nicht nur von OpenAI selbst gehostet, sondern auch von Azure.
Diese muss man auf dem Azure Portal selbst als Endpunkte konfigurieren, in der Regel leiden die OpenAI Azure Deployments weniger unter hoher Auslastung

### Andere Modelle
Auch wenn wir nicht damit arbeiten werden, ist es vielleicht ganz gut, die Namen der "großen" Konkurrenz-Modelle einmal gehört zu haben:
- `Gemini` Das neueste Google-Modell. Es hat den Fokus insbesondere auf multimodalem Input.
- `Claude` Claude ist die LLM-Reihe von Anthropic. Sehr viel Instruction-Tuning.
- `Mixtral` Das aktuell beste Open-Source Modell. Entwickelt von Mistral AI. Ein guter Kandidat für ein selbst gehostetes LLM.

### Temperatur
Alle LLMs sind nicht deterministisch. Aber die Temperatur ist ein Parameter, mit der man die Variabilität von Antworten hoch und runter schrauben kann.
Wie bei normalen Atomen ist die Bewegung niedrig, wenn die Temperatur niedrig ist. Wenn man die Temperatur hochschraubt, wird viel gewackelt.
Der Temperatur-Parameter ist üblicherweise ein Fließkommawert zwischen 0 und 1.

### Streaming
Nicht alle LLMs bieten die Möglichkeit, Token für Token live zu streamen. OpenAI-Modelle können es, man kann dies mit dem Streaming-Parameter einstellen.

### Links:
- https://www.langchain.com/
- https://python.langchain.com/docs/get_started/introduction
- https://platform.openai.com/docs/models/gpt-3-5
- https://platform.openai.com/docs/models/gpt-4
- https://platform.openai.com/docs/deprecations
- https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models

#### Wir definieren vier Modelle, je zwei von Azure und OpenAI, jeweils mit und ohne Streaming.

In [6]:
import os
from langchain_openai import AzureChatOpenAI, ChatOpenAI
openai_llm = ChatOpenAI(model=os.environ["OPENAI_MODEL"], temperature=0)
openai_llm_streaming = ChatOpenAI(model=os.environ["OPENAI_MODEL"], temperature=0.7, streaming=True)
azure_llm = AzureChatOpenAI(azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],temperature=0)
azure_llm_streaming = AzureChatOpenAI(azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],temperature=0.7,streaming=True)

#### In Langchain sind LLMs sehr leicht zur Laufzeit austauschbar. Das geht über die "configurable_alternatives". Die Syntax sieht so aus:

In [8]:
from langchain_core.runnables import ConfigurableField
llm = openai_llm.configurable_alternatives(ConfigurableField(id="llm"), default_key="openai",  openai_stream = openai_llm_streaming, azure=azure_llm, azure_stream=azure_llm_streaming)                                           

#### Jetzt kann man das LLM mit .with_config aufrufen. In dem Configurationsobjekt steht drin, welche Alternative man gerne hätte.

In [15]:
llm.with_config(configurable={"llm": "openai"}).invoke("Hi OpenAI! Kannst Du mir gerade mal einen hessischen Trinkspruch auf den Taunus im Dialekt erzeugen?")

AIMessage(content='Klar, hier ist ein hessischer Trinkspruch für dich, inspiriert vom Taunus:\n\n"Uff de Taunus, wo die Bäum so hoch,\nwo die Luft is frisch un\' der Himmel noch.\nWo die Wälder grün un\' die Felder weit,\nda trinke mer, in Freud un\' Leid.\nProst!"\n\nBeachte bitte, dass der Dialekt je nach Region im Taunus variieren kann.')

#### Jetzt nochmal mit Streaming und formatierung. Dazu rufen wir nicht invoke sondern astream auf (a für async). Wenn man die Zelle mehrmals ausführt, kommen andere Ergebnisse zurück (Temperatur != 0)

In [31]:
chunks = []
async for chunk in llm.with_config(configurable={"llm": "openai_stream"}).astream("Schreib mir doch bitte ein kleines Gedicht"):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

Natürlich, hier ist ein kleines Gedicht für dich:

Unter dem weiten, blauen Zelt,  
Wo Sterne funkeln, klar und hell,  
Da träumt die Welt in sanfter Ruh,  
Und leise schließt der Tag die Tür.  

Die Blumen neigen sich zur Nacht,  
Haben ihre Farbenpracht  
Dem Mondenschein leis' übergeben,  
Und nun im Traumland weiterleben.  

Der Wind, er singt ein Wiegenlied,  
Das durch die dunklen Wipfel zieht,  
Berührt die Seele zart und lind,  
Wiegt jeden Kummer fort im Wind.  

So legt sich Frieden übers Land,  
Gehalten von der Sterne Band,  
Und in der Stille, tief und weit,  
Findet das Herz zur Ruhe Zeit.  

Möge diese Nacht dich sanft umfangen,  
Deine Sorgen mit sich nehmen, ohne Verlangen.  
Und wenn der neue Tag erwacht,  
Sei erfüllt mit neuer Kraft und Macht.  

## Token

Token sind die kleinste Einheit des LLM.
Das LLM rechnet aus der Eingabe und den bisher errechneten Token die Wahrscheinlichkeit für den nächsten Token aus. Dieser neue Token wird dann angehängt und der nächste Token wird ermittelt.

So geht das immer weiter... bis das LLM glaubt, fertig zu sein. Auf diese Weise generieren LLMs die wahrschneinlichste Fortführung der Eingabetoken

Tokens können Wörter, machmal sogar Wortgruppen oder auch nur einzelne oder mehrere Buchstaben sein.

Die Bepreisung der LLMs ist an die Tokenanzahl (Eingabe und Ausgabe) gekoppelt.


Links:
- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb 

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

tokens = encoding.encode("AI ist eine tolle Sache.")

print(tokens)

decoded_tokens = [encoding.decode_single_token_bytes(token) for token in tokens]

print(decoded_tokens)

## Prompt Engineering und Templates in Langchain

Um die Dinge von der AI zu bekommen, die man erwartet, stellt man am besten sehr konkrete und präzise Anfragen.

Weil eine AI oft an ein bestimmtes Feld von Aufgaben gekoppelt ist, gibt man die Rahmenanweisung und Informationen dann in ein Template ein um nicht immer wieder die gleiche Rahmenanweisung zu schreiben.

Die jeweilige konkrete Nutzeranfrage wird dann in das Template eingefügt und das ausgefüllte Template ans LLM übergeben.

Der Trend immer mehr zu Chat-Modellen geht. Hierbei ist die Information, die man dem LLM gibt, in "Messages" unterteilt. Besondere Gewichtung hat eine System-Message. Diese kann Rahmenanweisungen enthalten, an die sich das LLM halten soll. Dem Nutzer wird es schwer fallen, das LLM dazu zu bewegen, sich über eine Anweisung in der System-Message hinweg zu setzen. Das LLM wurde ganz einfach darauf trainiert, sich an die Anweisungen einer System-Message unbedingt zu halten.

### Links
- https://python.langchain.com/docs/get_started/quickstart#prompt-templates
- https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/prompt-engineering
- https://learnprompting.org/docs/intro
- https://www.promptingguide.ai/
- https://smith.langchain.com/hub

In [25]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Du bist ein {beruf} aus Frankfurt."),
        ("human", "Erklär in 2 Sätzen im hessischen dialekt warum Deine Kunden aus {ort} die besten sind."),
    ]
)

print(prompt.format(beruf="Bäcker", ort="Bad Homburg"))


System: Du bist ein Bäcker aus Frankfurt.
Human: Erklär in 2 Sätzen im hessischen dialekt warum Deine Kunden aus Bad Homburg die besten sind.


### Langchain Hub Beispiel

Links:
- https://smith.langchain.com/hub/takatost/conversation-title-generator

In [26]:
from langchain import hub
template = hub.pull("takatost/conversation-title-generator")

print(template.format(question="Was kann ich mittels AI in meiner Firma automatisieren?"))

System: You need to decompose the user's input into "subject" and "intention" in order to accurately figure out what the user's input language actually is. 
Notice: the language type user use could be diverse, which can be English, Chinese, Español, Arabic, Japanese, French, and etc.
MAKE SURE your output is the SAME language as the user's input!
Your output is restricted only to: (Input language) Intention + Subject(short as possible)
Your output MUST be a valid JSON.

Tip: When the user's question is directed at you (the language model), you can add an emoji to make it more fun.


example 1:
User Input: hi, yesterday i had some burgers.
{
  "Language Type": "The user's input is pure English",
  "Your Reasoning": "The language of my output must be pure English.",
  "Your Output": "sharing yesterday's food"
}

example 2:
User Input: hello
{
  "Language Type": "The user's input is written in pure English",
  "Your Reasoning": "The language of my output must be pure English.",
  "Your Ou

## Und wieso heißt LangChain eigentlich LangChain?

### Links
- https://python.langchain.com/docs/modules/chains

In [34]:
import time
from langchain.chains import LLMChain
from langchain.schema import StrOutputParser # Hilft beim Formatieren

runnable = prompt | llm | StrOutputParser()
print(runnable.invoke({"beruf":"Schlachter", "ort":"Hanau"}))

Die Kundschaft aus Hanau, die is einfach die Beste, weil se immer treu bleibt und unser Fleisch wegen der Qualität schätzt. Unn dann, die komm' extra weche der guten Worscht und dem nette Babbel zu uns, des is doch was ganz Besonners.


In [35]:
# Streaming
async for chunk in runnable.with_config(configurable={"llm": "openai_stream"}).astream({"beruf":"Bäcker", "ort":"Würzburg"}):
    print(chunk, end="", flush=True)

Die Kundschaft aus Würzburg iss echt die beste, weil se immer so herzlich lache und unser Handkäs mit Musik so wertschätze. Un dann, wenn se unser Brot probiere, fühle se sich gleich wie dahoam, des is einfach wunnerbar!

## Embeddings (Textembeddings, Vektoren)

Beim Embedden wird ein Textstück genommen und in einen hochdimensionalen Vektor umgewandelt. Es gibt unterschiedliche Methoden, wie dieser Vektor erstellt wird und die genaue Art und Weise, wie der Embeddingprozess vonstatten geht, ist eine kleine Wissenschaft für sich.

**Allerdings haben alle Embeddings folgende Eigenschaft: Wenn zwei Textstücke ähnlichen Inhalt haben, liegen die resultierenden Vektoren im Raum nahe beieinander.**

**Man erhält durch den Embeddingprozess also eine Möglichkeit, sehr viel (!) Text, zusammen mit der Information, wie ähnlich diese Sätze inhaltlich sind, abzuspeichern!**

Vektoren können vom Computer sehr schnell verarbeitet werden. Man kann damit dann so schöne Dinge tun wie:
- Suche (wobei die Ergebnisse nach semantischer Relevanz geordnet werden)
- Clustering (wobei Textzeichenfolgen nach Ähnlichkeit gruppiert werden)
- Empfehlungen (wobei Elemente mit zugehörigen Textzeichenfolgen empfohlen werden)
- Anomalieerkennung (wobei Ausreißer mit geringem Zusammenhang identifiziert werden)
- Klassifizierung (wobei Textzeichenfolgen nach ihrer ähnlichsten Bezeichnung klassifiziert werden)

Die Standard Anwendung ist wie folgt:
- Alle Dokumente, die die Wissensdatenbank bilden (z.B. Betriebsanleitungen eines Maschinenherstellers) werden vektorisiert.
- Dann stellt ein Nutzer eine Anfrage an die KI (bezüglich einer Maschine).
- Diese Frage wird ebenfalls vektorisiert (das geht sehr schnell).
- Dann wird in den Vektoren der Betriebsanleitungen nach ähnlichen Vektoren gesucht (geht auch sehr schnell).
- Die ähnlichsten Vektoren werden rückaufgelöst (d.h. man sieht nach, welche ursprünglichen Dokumente hinter den Vektoren stehen).
- Diese Dokumente werden dann der AI als Kontext zum Beantworten der Frage mitzugeben.

Embeddings machen dies möglich, weil sie nicht auf der Grundlage von Zeichenfolgen arbeiten sondern wirklich eine semantische Nähe zueinander finden. Die Wörter "König" und "Prinz" sind sich im Vektorraum sehr ähnlich, obwohl die Wörter sich stark unterscheiden.

Es gibt sehr viele Embedding-Modelle, die für alle möglichen Fälle optimiert sind.
Modelle können einpsrachig oder mehrsprachig sein, wobei man für die mehrsprachigkeit einen Qualitätsverlust in Kauf nehmen muss!
Es gibt multimodale Embeddings die z.B. für das Wort Schraube und das Bild einer Schraube sehr ähnliche Vektoren herausgeben.

Links:
- https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
- https://python.langchain.com/docs/integrations/text_embedding/azureopenai
- https://app.twelvelabs.io/blog/multimodal-embeddings

In [41]:
from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(azure_deployment="textembeddingada002")
# embeddings = OpenAIEmbeddings()

text = "This is a test document."

query_result = embeddings.embed_query(text)

print("Dimensions: ",len(query_result))

# show the first 3 dimensions
print(query_result[:3])

Dimensions:  1536
[-0.0031584086849463177, 0.011094410093459925, -0.004001317253531179]


### Einfaches Beispiel von Entfernungsberechnung mittels Vektoren
Kleinere Zahl = Näher an der Referenz

In [ ]:
from langchain.evaluation import load_evaluator
import pandas as pd

from ipydatagrid import DataGrid, BarRenderer
from bqplot import LinearScale, ColorScale

# Two lists of sentences
reference = 'ICE'

sentences = [ 'Regionalbahn',
              'S-Bahn',
              'Nachtzug',
              'Flugzeug',
              'Icecream',
              'Zeppelin',
            ]

evaluator = load_evaluator("embedding_distance", embeddings=embeddings)

distances = []
for sentence in sentences:
    distance = evaluator.evaluate_strings(prediction=sentence, reference=reference)
    distances.append(distance["score"])


df = pd.DataFrame({
    "Satz": sentences,
    "Entfernung": distances
})

renderers = {
 "Entfernung": BarRenderer(
        horizontal_alignment="center",
        bar_color=ColorScale(min=0, max=1, scheme="viridis"),
        bar_value=LinearScale(min=0, max=1),
    )
}

grid = DataGrid(df, base_column_size=250, renderers=renderers)
grid.transform(
    [
        {"type": "sort", "columnIndex": 2, "desc": False},
    ]
)

grid

### Debug Informationen gewünscht?

In [ ]:
from langchain.globals import set_debug, set_verbose
set_debug(False)

In [ ]:
## Und jetzt selber mal Ausprobieren

In [ ]:
print(chain.run({
    'beruf': "Programmierer",
    'ort': "Stuttgart"
    }))